In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append('../src/')

%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import CIFAR10_utils as utils

# Tune hyperparameters

In [3]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [10, 100, 1000, 10000, 50000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
random_states = [1001, 2001, 3001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = 0
for lr_0, n, random_state, weight_decay in itertools.product(lr_0s, ns, random_states, weight_decays):
    count += 1
    model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(lr_0, n, random_state, weight_decay)
    print('    "python ../src/CIFAR10_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --random_state={} --tune --wandb --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, random_state, weight_decay))
print(count)

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=10_random_state=1001_weight_decay=0.01' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --weight_decay=0.01"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=10_random_state=1001_weight_decay=0.001' --n=10 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --tune --wandb --weight_decay=0.001"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-lear

In [4]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10'
lr_0s = np.logspace(-1, -4, num=4)
ns = [1000]
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'
prior_scales = np.logspace(0, 9, num=10)
random_states = [1001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
count = 0
for lr_0, n, prior_scale, random_state, weight_decay in itertools.product(lr_0s, ns, prior_scales, random_states, weight_decays):
    count += 1
    model_name = 'learned_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
    .format(lr_0, n, prior_scale, random_state, weight_decay)
    print('    "python ../src/CIFAR10_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --learned_prior --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --prior_scale={} --random_state={} --tune --wandb --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, prior_scale, random_state, weight_decay))
print(count)

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10' --learned_prior --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=1.0_random_state=1001_weight_decay=0.01' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --random_state=1001 --tune --wandb --weight_decay=0.01"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10' --learned_prior --lr_0=0.1 --model_name='learned_lr_0=0.1_n=1000_prior_scale=1.0_random_state=1001_weight_decay=0.001' --n=1000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --prior_scale=1.0 --random_state=1001 --tune --wandb --weight_decay=0.001"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tuft

# Retrain with selected parameters

In [5]:
def get_df(path):
    df = pd.read_csv(path, index_col='Unnamed: 0')
    return df

def get_val_nll(df, window_size):
    return np.mean(df.val_or_test_nll.values[-window_size:])

In [6]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10'
lr_0s = np.logspace(-1, -1, num=1)
ns = [10, 100, 1000, 10000, 50000]
random_states = [1001]
weight_decays = np.append(np.logspace(-2, -6, num=5), 0)
window_size = 3

In [7]:
columns = ['n', 'random_state', 'lr_0', 'weight_decay']
df = pd.DataFrame(columns=columns)

for n, random_state in itertools.product(ns, random_states):
    best_val_nll = np.inf
    best_hyperparameters = None
    for lr_0, weight_decay in itertools.product(lr_0s, weight_decays):
        model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
        .format(lr_0, n, random_state, weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        val_nll = get_val_nll(get_df(path), window_size)
        if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [n, random_state, lr_0, weight_decay]
    df.loc[df.shape[0]] = best_hyperparameters

In [8]:
df

,n,random_state,lr_0,weight_decay
0,10.0,1001.0,0.1,0.010
1,100.0,1001.0,0.1,0.010
2,1000.0,1001.0,0.1,0.010
3,10000.0,1001.0,0.1,0.001
4,50000.0,1001.0,0.1,0.001


In [172]:
dataset_path = '/cluster/tufts/hugheslab/eharve06/CIFAR-10'
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10'
prior_path = '/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior'

for row_index, row in df.iterrows():
    model_name = 'nonlearned_lr_0={}_n={}_random_state={}_weight_decay={}'\
    .format(row.lr_0, row.n, row.random_state, row.weight_decay)
    print('    "python ../src/CIFAR10_main.py --dataset_path=\'{}\' --experiments_path=\'{}\' --lr_0={} --model_name=\'{}\' --n={} --prior_path=\'{}\' --random_state={} --wandb --weight_decay={}"'\
          .format(dataset_path, experiments_path, lr_0, model_name, n, prior_path, random_state, weight_decay))

    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=10.0_random_state=1001.0_weight_decay=0.01' --n=50000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --wandb --weight_decay=0.0"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/retrained_CIFAR-10' --lr_0=0.1 --model_name='nonlearned_lr_0=0.1_n=100.0_random_state=1001.0_weight_decay=0.01' --n=50000 --prior_path='/cluster/tufts/hugheslab/eharve06/resnet50_ssl_prior' --random_state=1001 --wandb --weight_decay=0.0"
    "python ../src/CIFAR10_main.py --dataset_path='/cluster/tufts/hugheslab/eharve06/CIFAR-10' --experiments_path='/cluster/tufts/hugheslab/eharve06/bdl-transf

In [184]:
experiments_path = '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10'
lr_0s = np.logspace(-1, -1, num=1)
ns = [1000]
prior_scales = np.logspace(0, 0, num=1)
random_states = [1001]
weight_decays = np.logspace(-2, -2, num=1)
window_size = 3

In [185]:
columns = ['n', 'random_state', 'lr_0', 'prior_scale', 'weight_decay']
df = pd.DataFrame(columns=columns)

for n, random_state in itertools.product(ns, random_states):
    best_val_nll = np.inf
    best_hyperparameters = None
    for lr_0, prior_scale, weight_decay in itertools.product(lr_0s, prior_scales, weight_decays):
        model_name = 'learned_lr_0={}_n={}_prior_scale={}_random_state={}_weight_decay={}'\
        .format(lr_0, n, prior_scale, random_state, weight_decay)
        path =  '{}/{}.csv'.format(experiments_path, model_name)
        val_nll = get_val_nll(get_df(path), window_size)
        if val_nll < best_val_nll: best_val_nll = val_nll; best_hyperparameters = [n, random_state, lr_0, prior_scale, weight_decay]
    df.loc[df.shape[0]] = best_hyperparameters

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/tufts/hugheslab/eharve06/bdl-transfer-learning/experiments/tune_CIFAR-10/learned_lr_0=0.1_n=1000_prior_scale=1.0_random_state=1001_weight_decay=0.01.csv'

In [183]:
df

,n,random_state,lr_0,prior_scale,weight_decay
